## This file details the step by step analysis of the russian asat

In [4]:
from pathlib import Path
HERE = Path('.').resolve()
OUTPUT = HERE / 'output' / 'russian_asat'
OUTPUT.mkdir(exist_ok=True)
print(f'Using {OUTPUT} as output')

Using /home/danielk/git/antisat_analysis/projects/output/russian_asat as output


In [7]:
import sys
print(f'Adding {str(HERE.parent)} to path')
sys.path.append(str(HERE.parent))

Adding /home/danielk/git/antisat_analysis to path


In [20]:
data_paths = {
    'uhf': [
        '/home/danielk/data/spade/beamparks/uhf/2021.11.23',
        '/home/danielk/data/spade/beamparks/uhf/2021.11.25',
        '/home/danielk/data/spade/beamparks/uhf/2021.11.29',
    ],
    'esr': [
        '/home/danielk/data/spade/beamparks/esr/2021.11.19',
        '/home/danielk/data/spade/beamparks/esr/2021.11.23',
        '/home/danielk/data/spade/beamparks/esr/2021.11.25',
        '/home/danielk/data/spade/beamparks/esr/2021.11.29',
    ]
}

### Convert event.txt to h5

In [22]:
from convert_spade_events_to_h5 import main as conv_main

for key in data_paths:
    for pth in data_paths[key]:
        out = OUTPUT / key / Path(pth).name
        pth = Path(pth).resolve()
        if not out.parent.is_dir():
            out.parent.mkdir(parents=True, exist_ok=True)
        cmd = f'{str(pth)} {str(out)}'
        print(f'Running {cmd}')
        conv_main(cmd.split())


Running /home/danielk/data/spade/beamparks/uhf/2021.11.23 /home/danielk/git/antisat_analysis/projects/output/russian_asat/uhf/2021.11.23
/home/danielk/data/spade/beamparks/uhf/2021.11.23/leo_bpark_2.1u_NO@uhf/20211123_10/events.txt: Detected SPADE-file version 1.6...
/home/danielk/data/spade/beamparks/uhf/2021.11.23/leo_bpark_2.1u_NO@uhf/20211123_11/events.txt: Detected SPADE-file version 1.6...
/home/danielk/data/spade/beamparks/uhf/2021.11.23/leo_bpark_2.1u_NO@uhf/20211123_12/events.txt: Detected SPADE-file version 1.6...
/home/danielk/data/spade/beamparks/uhf/2021.11.23/leo_bpark_2.1u_NO@uhf/20211123_13/events.txt: Detected SPADE-file version 1.6...
/home/danielk/data/spade/beamparks/uhf/2021.11.23/leo_bpark_2.1u_NO@uhf/20211123_14/events.txt: Detected SPADE-file version 1.6...
/home/danielk/data/spade/beamparks/uhf/2021.11.23/leo_bpark_2.1u_NO@uhf/20211123_15/events.txt: Detected SPADE-file version 1.6...
/home/danielk/data/spade/beamparks/uhf/2021.11.23/leo_bpark_2.1u_NO@uhf/20211

/home/danielk/venvs/SORTS_applications/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


/home/danielk/data/spade/beamparks/uhf/2021.11.29/leo_bpark_2.1u_SW@uhf/20211129_10/events.txt: Detected SPADE-file version 1.6...
/home/danielk/data/spade/beamparks/uhf/2021.11.29/leo_bpark_2.1u_SW@uhf/20211129_11/events.txt: Detected SPADE-file version 1.6...
/home/danielk/data/spade/beamparks/uhf/2021.11.29/leo_bpark_2.1u_SW@uhf/20211129_12/events.txt: Detected SPADE-file version 1.6...
/home/danielk/data/spade/beamparks/uhf/2021.11.29/leo_bpark_2.1u_SW@uhf/20211129_13/events.txt: Detected SPADE-file version 1.6...
/home/danielk/data/spade/beamparks/uhf/2021.11.29/leo_bpark_2.1u_SW@uhf/20211129_14/events.txt: Detected SPADE-file version 1.6...
/home/danielk/data/spade/beamparks/uhf/2021.11.29/leo_bpark_2.1u_SW@uhf/20211129_15/events.txt: Detected SPADE-file version 1.6...
/home/danielk/data/spade/beamparks/uhf/2021.11.29/leo_bpark_2.1u_SW@uhf/20211129_16/events.txt: Detected SPADE-file version 1.6...
/home/danielk/data/spade/beamparks/uhf/2021.11.29/leo_bpark_2.1u_SW@uhf/20211129_17

Adding row 26
Adding row 27
Adding row 28
Adding row 29
Skipping row 30: rt**2.0 = sn = 29.160000000000004 <= 33
Adding row 31
Adding row 32
Skipping row 33: rt**2.0 = sn = 31.359999999999996 <= 33
Adding row 34
Adding row 35
Skipping row 36: rt**2.0 = sn = 26.009999999999998 <= 33
Skipping row 37: rt**2.0 = sn = 30.25 <= 33
Adding row 38
Adding row 39
Adding row 40
Skipping row 41: rt**2.0 = sn = 26.009999999999998 <= 33
Adding row 42
Adding row 43
Skipping row 44: rt**2.0 = sn = 28.09 <= 33
Adding row 45
Adding row 46
Adding row 47
Adding row 48
Skipping row 49: rt**2.0 = sn = 28.09 <= 33
Skipping row 50: rt**2.0 = sn = 25.0 <= 33
Adding row 51
Adding row 52
Adding row 53
Adding row 54
Adding row 55
Adding row 0
Adding row 1
Adding row 2
Adding row 3
Adding row 4
Adding row 5
Adding row 6
Adding row 7
Adding row 8
Adding row 9
Adding row 10
Adding row 11
Adding row 12
Skipping row 13: rt**2.0 = sn = 25.0 <= 33
Skipping row 14: rt**2.0 = sn = 30.25 <= 33
Adding row 15
Adding row 16
Ad

Adding row 61
Adding row 62
Adding row 63
Adding row 64
Adding row 65
Adding row 66
Adding row 67
Adding row 68
Adding row 69
Adding row 70
Adding row 0
Adding row 1
Adding row 2
Adding row 3
Adding row 4
Adding row 5
Adding row 6
Adding row 7
Skipping row 8: rt**2.0 = sn = 28.09 <= 33
Skipping row 9: rt**2.0 = sn = 25.0 <= 33
Adding row 10
Adding row 11
Adding row 12
Adding row 13
Adding row 14
Adding row 15
Adding row 16
Adding row 17
Adding row 18
Adding row 19
Adding row 20
Adding row 21
Adding row 22
Adding row 23
Adding row 24
Adding row 25
Adding row 26
Adding row 27
Adding row 28
Adding row 29
Adding row 30
Adding row 31
Adding row 32
Adding row 33
Adding row 34
Adding row 35
Adding row 36
Adding row 37
Adding row 38
Adding row 39
Adding row 40
Adding row 41
Adding row 42
Adding row 43
Adding row 44
Adding row 45
Adding row 46
Adding row 47
Adding row 48
Adding row 49
Adding row 50
Adding row 51
Adding row 52
Adding row 53
Adding row 54
Adding row 55
Adding row 56
Adding row 57

Adding row 85
Adding row 86
Adding row 87
Adding row 88
Adding row 89
Adding row 90
Adding row 91
Adding row 92
Adding row 93
Adding row 94
Adding row 95
Adding row 96
Adding row 97
Adding row 98
Skipping row 99: rt**2.0 = sn = 31.359999999999996 <= 33
Adding row 100
Adding row 101
Skipping row 102: rt**2.0 = sn = 29.160000000000004 <= 33
Adding row 103
Adding row 104
Adding row 105
Adding row 106
Adding row 107
Adding row 108
Adding row 109
Skipping row 110: rt**2.0 = sn = 25.0 <= 33
Adding row 111
Skipping row 112: rt**2.0 = sn = 26.009999999999998 <= 33
Adding row 113
Adding row 114
Adding row 115
Adding row 116
Adding row 0
Adding row 1
Adding row 2
Adding row 3
Adding row 4
Adding row 5
Adding row 6
Adding row 7
Adding row 8
Adding row 9
Adding row 10
Adding row 11
Adding row 12
Adding row 13
Adding row 14
Adding row 15
Adding row 16
Adding row 17
Adding row 18
Adding row 19
Adding row 20
Adding row 21
Adding row 22
Adding row 23
Adding row 24
Adding row 25
Adding row 26
Adding ro

/home/danielk/data/spade/beamparks/esr/2021.11.29/leo_bpark_2.2_SW@42m/20211129_08/events.txt: Detected SPADE-file version 1.6...
/home/danielk/data/spade/beamparks/esr/2021.11.29/leo_bpark_2.2_SW@42m/20211129_09/events.txt: Detected SPADE-file version 1.6...
/home/danielk/data/spade/beamparks/esr/2021.11.29/leo_bpark_2.2_SW@42m/20211129_10/events.txt: Detected SPADE-file version 1.6...
/home/danielk/data/spade/beamparks/esr/2021.11.29/leo_bpark_2.2_SW@42m/20211129_11/events.txt: Detected SPADE-file version 1.6...
/home/danielk/data/spade/beamparks/esr/2021.11.29/leo_bpark_2.2_SW@42m/20211129_12/events.txt: Detected SPADE-file version 1.6...
/home/danielk/data/spade/beamparks/esr/2021.11.29/leo_bpark_2.2_SW@42m/20211129_13/events.txt: Detected SPADE-file version 1.6...
/home/danielk/data/spade/beamparks/esr/2021.11.29/leo_bpark_2.2_SW@42m/20211129_14/events.txt: Detected SPADE-file version 1.6...
/home/danielk/data/spade/beamparks/esr/2021.11.29/leo_bpark_2.2_SW@42m/20211129_15/events.